## AWS y Intel Hackathon: Model Training

### Install Python SDKs

In [1]:
import sys

In [ ]:
!{sys.executable} -m pip install sagemaker-experiments==0.1.24

### Install PyTroch

In [ ]:
# pytorch version needs to be the same in both the notebook instance and the training job container
# https://github.com/pytorch/pytorch/issues/25214
!{sys.executable} -m pip install torch==1.1.0
!{sys.executable} -m pip install torchvision==0.3.0
!{sys.executable} -m pip install pillow==6.2.2
!{sys.executable} -m pip install --upgrade sagemaker
!{sys.executable} -m pip install torchsummary

### Setup

In [ ]:
import time

import boto3
import numpy as np
import pandas as pd
from IPython.display import set_matplotlib_formats, display
from matplotlib import pyplot as plt
from torchvision import datasets, transforms, models

import torch

import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import Session
from sagemaker.analytics import ExperimentAnalytics

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

from tqdm.notebook import tqdm

from torchsummary import summary
import glob
from PIL import Image

import random

set_matplotlib_formats("retina")

### Download the data

In [ ]:
!wget https://www.dropbox.com/s/kc4xt9fhkrdnwjo/dataset_reduced.zip

Para instancias con menos HW puede ser usada tambien la version reducida del dataset, con aproximadamente 200 imagenes por clase. Para ello ejecutar la siguiente celda

In [ ]:
!wget https://www.dropbox.com/s/lu3ftca48y92x1q/final_dataset.zip

### Upload dataset to S3 as zip file

In [31]:
sm_sess = sagemaker.Session()
sess = sm_sess.boto_session
sm = sm_sess.sagemaker_client
role = get_execution_role()

In [ ]:
# create a s3 bucket to hold data, note that your account might already created a bucket with the same name
account_id = sess.client("sts").get_caller_identity()["Account"]
bucket = "sagemaker-hackathon-demo-{}-{}".format(sess.region_name, account_id)
prefix = "hackathon"

try:
    if sess.region_name == "us-east-1":
        sess.client("s3").create_bucket(Bucket=bucket)
    else:
        sess.client("s3").create_bucket(
            Bucket=bucket, CreateBucketConfiguration={"LocationConstraint": sess.region_name}
        )
except Exception as e:
    print(e)

In [ ]:
bucket

In [ ]:
s3_resource = boto3.resource("s3", region_name = sess.region_name)

inputs = None

try:
#     inputs = s3_resource.meta.client.upload_file("./dataset_reduced.zip", bucket, "hackathon/dataset_reduced.zip")
#     print("inputs", inputs)
    
    inputs = sagemaker.Session().upload_data(path="./dataset_reduced.zip", bucket=bucket, key_prefix=prefix)
    print("input spec: {}".format(inputs))
except Exception as exp:
    print("exp: ", exp)


In [51]:
# sagemaker.Session().download_file(bucket,"dataset_reduced.zip",'./dataset_reduced_demo.zip')

s3_client = boto3.client("s3")
s3_client.download_file(
    bucket,
    "hackathon/dataset_reduced.zip",
    "dataset_reduced_2.zip",
)

### Prepare the dataset

In [ ]:
!unzip -uo dataset_reduced.zip

In [10]:
root_path = "./final_dataset/"

In [11]:
img_transform= {
    'dataset':
    transforms.Compose([
        transforms.Resize((505, 505)), # Mean size of images is 505
        transforms.ToTensor(),
    ]),
    'training':
    transforms.Compose([
         transforms.RandomResizedCrop(size=224),
         transforms.RandomHorizontalFlip(),
    ]),
    'validation': 
    transforms.Compose([
        transforms.Resize(size=224),
    ]),
    'testing': 
    transforms.Compose([
        transforms.Resize((505, 505)),
        transforms.ToTensor(),
    ]),
}

In [ ]:
train_dataset = datasets.ImageFolder(root_path + "train/", transform=img_transform['dataset'])
print('#{} train images loaded succesfully!'.format(len(train_dataset)))

In [ ]:
test_dataset = datasets.ImageFolder(root_path + "GT/", transform=img_transform['dataset'])
print('#{} test images loaded succesfully!'.format(len(test_dataset)))

In [ ]:
for i in range(10):
    n = random.randrange(len(train_dataset))
    print("n", n)

    print("Label = ", train_dataset[n][1] + 1, ":")
    display(transforms.ToPILImage()(train_dataset[n][0]))

In [ ]:
for i in range(10):
    n = random.randrange(len(test_dataset))
    print("n", n)

    print("Label = ", test_dataset[n][1] + 1, ":")
    display(transforms.ToPILImage()(test_dataset[n][0]))

In [ ]:
SPLIT_SIZE = .1
n_val = round(SPLIT_SIZE * len(train_dataset))

train_set, val_set = torch.utils.data.random_split(train_dataset, [len(train_dataset)-n_val, n_val])

print('Spliting train images to #{} training samples and #{} samples for validation'.format(len(train_set), len(val_set)))

### Dataset
We download the MNIST hand written digits dataset, and then apply transformation on each of the image.

After transforming the images in the dataset, we upload it to s3.

Now lets track the parameters from the data pre-processing step.

In [42]:
with Tracker.create(display_name="Preprocessing", sagemaker_boto_client=sm) as tracker:
    tracker.log_parameters(
        {
            "normalization_mean": 0.1307,
            "normalization_std": 0.3081,
        }
    )
#     # we can log the s3 uri to the dataset we just uploaded
#     tracker.log_input(name="mnist-dataset", media_type="s3/uri", value=inputs)

### Step 1 - Set up the Experiment

Create an experiment to track all the model training iterations. Experiments are a great way to organize your data science work. You can create experiments to organize all your model development work for : [1] a business use case you are addressing (e.g. create experiment named “customer churn prediction”), or [2] a data science team that owns the experiment (e.g. create experiment named “marketing analytics experiment”), or [3] a specific data science and ML project. Think of it as a “folder” for organizing your “files”.

### Create an Experiment

In [ ]:
model_experiment = Experiment.create(
    experiment_name=f"hackathon-model-experiment-{int(time.time())}",
    description="Classification of trash in images into containers",
    sagemaker_boto_client=sm,
)
print(model_experiment)

### Step 2 - Track Experiment
### Now create a Trial for each training run to track the it's inputs, parameters, and metrics.
While training the CNN model on SageMaker, we will experiment with several values for the number of hidden channel in the model. We will create a Trial to track each training job run. We will also create a TrialComponent from the tracker we created before, and add to the Trial. This will enrich the Trial with the parameters we captured from the data pre-processing stage.

Note the execution of the following code takes a while.

In [44]:
from sagemaker.pytorch import PyTorch, PyTorchModel

In [45]:
hidden_channel_trial_name_map = {}

If you want to run the following training jobs asynchronously, you may need to increase your resource limit. Otherwise, you can run them sequentially.

In [46]:
preprocessing_trial_component = tracker.trial_component

In [ ]:
# [2, 5, 10, 20, 32]

for i, num_hidden_channel in enumerate([2, 5]):
    # create trial
    trial_name = f"cnn-training-job-{num_hidden_channel}-hidden-channels-{int(time.time())}"
    cnn_trial = Trial.create(
        trial_name=trial_name,
        experiment_name=model_experiment.experiment_name,
        sagemaker_boto_client=sm,
    )
    hidden_channel_trial_name_map[num_hidden_channel] = trial_name

    # associate the proprocessing trial component with the current trial
    cnn_trial.add_trial_component(preprocessing_trial_component)

    # all input configurations, parameters, and metrics specified in estimator
    # definition are automatically tracked
    estimator = PyTorch(
        py_version="py3",
        entry_point="./model.py",
        role=role,
        sagemaker_session=sagemaker.Session(sagemaker_client=sm),
        framework_version="1.1.0",
        instance_count=1,
        instance_type="ml.m4.xlarge",
        hyperparameters={
            "epochs": 2,
            "backend": "gloo",
            "hidden_channels": num_hidden_channel,
            "dropout": 0.2,
            "kernel_size": 5,
            "optimizer": "sgd",
        },
        metric_definitions=[
            {"Name": "train:loss", "Regex": "Train Loss: (.*?);"},
            {"Name": "test:loss", "Regex": "Test Average loss: (.*?),"},
            {"Name": "test:accuracy", "Regex": "Test Accuracy: (.*?)%;"},
        ],
        enable_sagemaker_metrics=True,
    )

    cnn_training_job_name = "cnn-training-job-{}".format(int(time.time()))

    # Now associate the estimator with the Experiment and Trial
    estimator.fit(
        inputs={"training": inputs},
        job_name=cnn_training_job_name,
        experiment_config={
            "TrialName": cnn_trial.trial_name,
            "TrialComponentDisplayName": "Training",
        },
        wait=True,
    )

    # give it a while before dispatching the next training job
    time.sleep(2)

## Deploy endpoint for the best training-job / trial component

Now we'll take the best (as sorted) and create an endpoint for it.

In [ ]:
# Pulling best based on sort in the analytics/dataframe so first is best....
best_trial_component_name = trial_component_analytics.dataframe().iloc[0]["TrialComponentName"]
best_trial_component = TrialComponent.load(best_trial_component_name)

model_data = best_trial_component.output_artifacts["SageMaker.ModelArtifact"].value
env = {
    "hidden_channels": str(int(best_trial_component.parameters["hidden_channels"])),
    "dropout": str(best_trial_component.parameters["dropout"]),
    "kernel_size": str(int(best_trial_component.parameters["kernel_size"])),
}
model = PyTorchModel(
    model_data,
    role,
    "./deploy-ei.py",
    py_version="py3",
    env=env,
    sagemaker_session=sagemaker.Session(sagemaker_client=sm),
    framework_version="1.1.0",
    name=best_trial_component.trial_component_name,
)

predictor = model.deploy(instance_type="ml.m4.xlarge", initial_instance_count=1)

## Cleanup

Once we're doing don't forget to clean up the endpoint to prevent unnecessary billing.

> Trial components can exist independent of trials and experiments. You might want keep them if you plan on further exploration. If so, comment out tc.delete()

In [ ]:
predictor.delete_endpoint()